In [1]:
%load_ext autoreload
%autoreload 2

这个notebook包括所有执行代码，主要逻辑则包括再action_machine.py, event_manager.py, novel_helper.py等几个python文件里

In [2]:
import os
import sys
import random
import math
import collections
import logging
import pandas as pd
import numpy as np

from proto.book_def_pb2 import Book as BookDef
from proto.book_def_pb2 import BookWarehouse
from proto.user_def_pb2 import Robot as RobotDef
from proto.user_def_pb2 import RobotRegiment
from proto.user_def_pb2 import RobotArmy

from novel_helper import NovelHelper
from novel_helper import BookWarehouseDumper
from novel_helper import S

from enlighten_helper import EnlightenHelper

from action_machine import ActionMachine
from event_manager import EventManager

BookWarehouseDumper用于将爬虫抓的语料生成book_def.proto中的格式

In [3]:
# book_warehouse_dumper = BookWarehouseDumper()

从fictiontg网站上抓下来的有3万余篇，book_meta_10t.csv文件存储着其中1万篇小说，这样可以保证候选的稳定性。如果确有需要，可以通过以下方式重新生成这个csv文件。注意这个是双下划线的方法，这是为了防止误操作。这样的好处是之后可以用另外的1万本小说来测试系统的效果，看一些现象是否还能复现。

In [4]:
# book_warehouse_dumper.__gen_10t_csv()

生成book_warehouse，这个过程包括切句切词，1万篇小说要用20分钟以上。结果存储在book_warehouse_1t.bin和book_warehouse_10t.bin两个二进制文件中

In [5]:
# book_warehouse_dumper.dump_book_warehouse()

从.bin文件加载BookWarehouse进内存，默认是1千篇，可以通过参数改成1万篇

In [6]:
book_warehouse_type = '1t'
book_warehouse = NovelHelper.load_book_warehouse(book_warehouse_type)

生成meta.pkl

In [24]:
# from novel_helper import MetaManager
# meta_manager = MetaManager(book_warehouse)
# meta_manager.dump_meta()

 1000

验证meta.pkl

In [26]:
# meta_manager = MetaManager()
# k, c, t = meta_manager.load_meta()
# len(t)
# len(c)
# len(t)
# list(c.keys())

生成NovelHelper，用于辅助统计与词向量训练，需要传入book_warehouse。会同时load之前存好的word_book_count_map和词向量数据

In [35]:
novel_helper = NovelHelper(book_warehouse=book_warehouse)

一次性的fetch所有较高频单词的汉语翻译

In [37]:
# novel_helper._fetch_cn_online()

 2 thirteen 十三                     list index out of range
list index out of range
 2904 murmurs 杂音                                list index out of range
 7920 tern 燕鸥                                      list index out of range
 9594 eventuality 可能性                            

统计每个单词在多少篇小说里出现了，结果存到word_book_count.csv中

In [ ]:
# novel_helper.dump_word_book_count_csv()

训练词向量（如有需要），会先生成训练语料到wv_corpus.txt，进而训练词向量文件到wv.bin，大约用10分钟

In [ ]:
# novel_helper.train_and_dump_wv_bin()

检查词向量效果-相似词

In [14]:
# r=novel_helper.get_similar_word('fuck')
# r

检查词向量效果-类比词

In [19]:
# r=novel_helper.get_analogy_word('father', 'fuck', 'mother')
# r

查看所有的keyword、category和rating

In [76]:
# keyword_bid_set_map, category_bid_set_map = collections.defaultdict(set), collections.defaultdict(set)
# rating_bid_set_map = collections.defaultdict(set)
# for book in book_warehouse.book:
#     for keyword in book.keyword:
#         keyword_bid_set_map[keyword.lower()].add(book.bid)
#     for category in book.category:
#         category_bid_set_map[category.lower()].add(book.bid)
#     rating_bid_set_map[book.rating].add(book.bid)
# category_book_count_map = {k.lower():len(category_bid_set_map[k]) for k in category_bid_set_map}
# keyword_book_count_map = {k.lower():len(keyword_bid_set_map[k]) for k in keyword_bid_set_map}
# rating_book_count_map = {k.lower():len(rating_bid_set_map[k]) for k in rating_bid_set_map}
# category_data = list(map(lambda x: {'category': x[0].lower(), 'c_book_count': x[1]}, category_book_count_map.items()))
# keyword_data = list(map(lambda x: {'keyword': x[0].lower(), 'k_book_count': x[1]}, keyword_book_count_map.items()))
# rating_data = list(map(lambda x: {'rating': x[0].lower(), 'r_book_count': x[1]}, rating_book_count_map.items()))
# category_data.sort(key=lambda x: x['c_book_count'], reverse=True)
# keyword_data.sort(key=lambda x: x['k_book_count'], reverse=True)
# rating_data.sort(key=lambda x: x['r_book_count'], reverse=True)
# df_category = pd.DataFrame(category_data)
# df_keyword = pd.DataFrame(keyword_data)
# df_rating = pd.DataFrame(rating_data)
# pd.set_option('display.max_rows', 500) # 打印最大行数
# df_category
# # df_rating
# # df_keyword

将带有某些特征的book以人类可读的方式dump到library里

In [80]:
# bids = list(category_bid_set_map['crime punishment'])
# for book in book_warehouse.book:
#     if book.bid in bids:
#         novel_helper.dump_book_to_library(book)

EnlighterHelper，用于生成并enlighten（启蒙）机器人用户大军

In [7]:
enlighten_helper = EnlightenHelper(robot_army = None, novel_helper=novel_helper)

开始生产RobotUser

In [8]:
# enlighten_helper.enlighten_and_dump()

加载训练好的RobotArmy

In [39]:
# robot_army = EnlightenHelper.load_robot_army()

检查机器人军团整体情况

In [18]:
# enlighten_helper.report_robot_army(robot_army)

regiment_count: 10,robot_count: 10000


检查robot信息（机器人秘书）

In [85]:
# robot_army_secret = robot_army.regiment[0].robot[6]
# enlighten_helper.get_robot_repr(robot_army_secret)

初始化一个ActionMachine

In [180]:
action_machine = ActionMachine(novel_helper=novel_helper, robot_army=robot_army)

测试`_get_click_drive_score`,`_get_read_drive_score`、`_get_detest_drive_score`

In [179]:
# book_x = book_warehouse.book[0]
# robot_user_x = robot_army.regiment[0].robot[0]
# chapter = 3
# click_drive_score = action_machine._get_click_drive_score(robot_user_x, book_x, sample_loop=2)
# read_drive_score = action_machine._get_read_drive_score(robot_user_x, book_x, chapter, sample_loop=2)
# detest_title_drive_score = action_machine._get_title_detest_drive_score(robot_user_x, book_x, sample_loop=2)
# detest_drive_score = action_machine._get_detest_drive_score(robot_user_x, book_x, chapter, sample_loop=2)
# addict_drive_score = action_machine._get_addict_drive_score(robot_user_x, book_x, chapter, sample_loop=2)

# print(click_drive_score, read_drive_score, detest_title_drive_score, detest_drive_score, addict_drive_score)

0.06187081848340892 0.02144914949985688 0.01875598491960219 -0.01981412640888699 0.040873518941113046


运行30个period

In [231]:
action_machine = ActionMachine(novel_helper=novel_helper, robot_army=robot_army)
action_machine.run(300)

ERROR:root:period: 1,offline: 9898,recommend: 102,choosing: 0,click_drive: 0,read_drive: 0,reading: 0
ERROR:root:period: 2,offline: 9884,recommend: 14,choosing: 102,click_drive: 0,read_drive: 0,reading: 0
ERROR:root:period: 3,offline: 9851,recommend: 33,choosing: 116,click_drive: 0,read_drive: 0,reading: 0
ERROR:root:period: 4,offline: 9819,recommend: 32,choosing: 149,click_drive: 0,read_drive: 0,reading: 0
ERROR:root:period: 5,offline: 9787,recommend: 32,choosing: 79,click_drive: 102,read_drive: 0,reading: 0
ERROR:root:period: 6,offline: 9731,recommend: 58,choosing: 97,click_drive: 14,read_drive: 100,reading: 0
ERROR:root:period: 7,offline: 9671,recommend: 60,choosing: 122,click_drive: 33,read_drive: 14,reading: 100
ERROR:root:period: 8,offline: 9601,recommend: 71,choosing: 150,click_drive: 32,read_drive: 32,reading: 114
ERROR:root:period: 9,offline: 9521,recommend: 81,choosing: 189,click_drive: 32,read_drive: 31,reading: 146
ERROR:root:period: 10,offline: 9447,recommend: 76,choosing:

ERROR:root:period: 76,offline: 3066,recommend: 94,choosing: 445,click_drive: 261,read_drive: 1431,reading: 4703
ERROR:root:period: 77,offline: 2971,recommend: 101,choosing: 431,click_drive: 247,read_drive: 1434,reading: 4816
ERROR:root:period: 78,offline: 2867,recommend: 107,choosing: 444,click_drive: 247,read_drive: 1419,reading: 4916
ERROR:root:period: 79,offline: 2787,recommend: 85,choosing: 448,click_drive: 249,read_drive: 1410,reading: 5021
ERROR:root:period: 80,offline: 2684,recommend: 110,choosing: 425,click_drive: 240,read_drive: 1423,reading: 5118
ERROR:root:period: 81,offline: 2584,recommend: 104,choosing: 452,click_drive: 233,read_drive: 1528,reading: 5099
ERROR:root:period: 82,offline: 2494,recommend: 95,choosing: 431,click_drive: 257,read_drive: 1503,reading: 5220
ERROR:root:period: 83,offline: 2397,recommend: 103,choosing: 487,click_drive: 217,read_drive: 1524,reading: 5272
ERROR:root:period: 84,offline: 2296,recommend: 109,choosing: 439,click_drive: 288,read_drive: 1487,

ERROR:root:period: 150,offline: 168,recommend: 6,choosing: 259,click_drive: 203,read_drive: 1996,reading: 7368
ERROR:root:period: 151,offline: 169,recommend: 6,choosing: 229,click_drive: 248,read_drive: 2083,reading: 7265
ERROR:root:period: 152,offline: 168,recommend: 5,choosing: 248,click_drive: 219,read_drive: 2030,reading: 7330
ERROR:root:period: 153,offline: 170,recommend: 2,choosing: 243,click_drive: 236,read_drive: 2093,reading: 7256
ERROR:root:period: 154,offline: 171,recommend: 1,choosing: 245,click_drive: 232,read_drive: 2051,reading: 7300
ERROR:root:period: 155,offline: 169,recommend: 4,choosing: 230,click_drive: 238,read_drive: 2014,reading: 7345
ERROR:root:period: 156,offline: 164,recommend: 8,choosing: 232,click_drive: 227,read_drive: 2086,reading: 7283
ERROR:root:period: 157,offline: 162,recommend: 6,choosing: 232,click_drive: 227,read_drive: 2027,reading: 7346
ERROR:root:period: 158,offline: 158,recommend: 6,choosing: 239,click_drive: 220,read_drive: 2086,reading: 7291
E

ERROR:root:period: 224,offline: 97,recommend: 1,choosing: 245,click_drive: 264,read_drive: 2084,reading: 7309
ERROR:root:period: 225,offline: 96,recommend: 2,choosing: 270,click_drive: 240,read_drive: 2057,reading: 7335
ERROR:root:period: 226,offline: 96,recommend: 1,choosing: 253,click_drive: 265,read_drive: 2089,reading: 7296
ERROR:root:period: 227,offline: 96,recommend: 1,choosing: 265,click_drive: 250,read_drive: 2110,reading: 7278
ERROR:root:period: 228,offline: 96,recommend: 1,choosing: 280,click_drive: 262,read_drive: 2068,reading: 7293
ERROR:root:period: 229,offline: 95,recommend: 2,choosing: 233,click_drive: 278,read_drive: 2081,reading: 7311
ERROR:root:period: 230,offline: 94,recommend: 2,choosing: 240,click_drive: 231,read_drive: 2088,reading: 7345
ERROR:root:period: 231,offline: 93,recommend: 2,choosing: 269,click_drive: 237,read_drive: 2058,reading: 7341
ERROR:root:period: 232,offline: 93,recommend: 1,choosing: 223,click_drive: 265,read_drive: 2070,reading: 7348
ERROR:root

ERROR:root:period: 299,offline: 60,recommend: 2,choosing: 279,click_drive: 254,read_drive: 2056,reading: 7349
ERROR:root:period: 300,offline: 62,recommend: 1,choosing: 271,click_drive: 277,read_drive: 2035,reading: 7354


统计点击率

In [221]:
ctr_sql = """select 1.0 * sum(case when t2.uid is not null then 1 else 0 end)/sum(1) as ctr, sum(1) as impr_cnt from
(select uid, bid from novel_event where event = 'impression') t1 left join
(select uid, bid from novel_event where event = 'go_detail') t2 on t1.uid = t2.uid and t1.bid = t2.bid
"""
read_count_per_book_sql = """
select bid, count(*) as read_cnt from novel_event where event = 'go_detail' 
group by bid order by read_cnt desc limit 1000
"""
how_many_book_sql = """
select count(distinct bid) from novel_event
"""
action_machine.query(ctr_sql, reprepare=False)


[(0.0, 311128)]

In [137]:
from action_machine import UbInfo
ubi = UbInfo(1,2)
score = 0.21
ubi.next_chapter(score)
ubi

start_period=1, chapter_start=2, chapter_count=1